In [28]:
!pip install -U scikit-learn==1.2.2

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [30]:
from google.colab import drive
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import label_binarize, StandardScaler
from sklearn.metrics import roc_curve, auc
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif, chi2

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
folder_path = '/content/drive/MyDrive/Codigo Tesis Helicobacter Editado/Var_Prev'
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

In [32]:
## Lista para almacenar dataframes
results = []
sm = SMOTE(random_state=42)

for file in csv_files:

    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path, sep=",", header=0, index_col=0)
    df = df.dropna()
    df = df.sample(n=500, replace=True, random_state=42)
    X = df.drop("Prevalencia", axis=1)
    y = df["Prevalencia"]

    # Determinar 'k', el número de variables
    k = X.shape[1]

    pipeline = Pipeline([
        ('scaler', StandardScaler()),    # Escalado de características
        ('reduce_dim', 'passthrough'),   # Placeholder para la reducción de dimensionalidad
        ('classification',MLPClassifier(random_state=42, verbose=False))
    ])

    # Definir los parámetros para GridSearch
    param_grid = [
        {
            'reduce_dim': [SelectKBest(score_func=f_classif),
                SelectKBest(score_func=mutual_info_classif),
                SelectKBest(score_func=chi2)],
            'reduce_dim__k': [k//2,k//4,k//8],
            'classification__learning_rate_init': [0.01,0.001,0.1],
            'classification__max_iter': [2000],
            'classification__momentum': [0.6,0.62,0.64],
            'classification__solver': ['adam'],
            'classification__hidden_layer_sizes':[ (4*k, 2*k),(2*k, 2*k),
             (4*k, 4*k),(4*k, 2*k,2*k),]
        },
        {
            'reduce_dim': [PCA(random_state=42)],
            'reduce_dim__n_components': [k//2,k//4,k//8],
            'classification__learning_rate_init': [0.01,0.001,0.1],
            'classification__max_iter': [2000],
            'classification__momentum': [0.6,0.62,0.64],
            'classification__solver': ['adam'],
            'classification__hidden_layer_sizes':[ (4*k, 2*k),(2*k, 2*k),
             (4*k, 4*k),(4*k, 2*k,2*k),]
        }
    ]

    X_res, y_res = sm.fit_resample(X, y)
    X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.332,
                                                        stratify=y_res, random_state=42)

    # Configurar el clasificador con diferentes tamaños de capa oculta
    clf = GridSearchCV(pipeline, param_grid=param_grid, cv=3, scoring='accuracy')

    # Configurar el clasificador con diferentes tamaños de capa oculta

    clf.fit(X_train, y_train)

    test_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, test_pred)

    results.append((file, clf.best_params_, accuracy))

,MaterialTecho,Habitantes/Vivienda,Humedades,Temperatura,Altitud_Vereda,Uso_Recipiente_Para_Agua,Medidas_Contra_HP,Area_HA_Vereda,Viviendas_Veredas,Hogares_Vereda,...,Sedimentos,LavadoAntesComer,Ducha,Basura,MaterialPiso,MaterialPared,Tipo_Sanitarios,Cantidad_Sanitarios,LavadoPostSanitario,Etnia
0,0.000000,6.444440,93.000000,14.450000,1851.222220,0.888890,0.111110,1753.938160,536.000000,592.000000,...,0.000000,1.000000,1.000000,4.000000,3.0,2.000000,1.000000,1.000000,1.000000,3.000000
1,0.000000,4.250000,95.000000,9.300000,2917.500000,0.500000,0.000000,7594.682830,91.000000,93.000000,...,0.000000,0.750000,1.000000,1.000000,3.0,2.000000,1.000000,1.000000,1.000000,3.000000
2,0.000000,6.785710,93.000000,13.940000,1869.785710,1.000000,0.071430,1582.076640,305.500000,340.500000,...,0.000000,1.000000,0.928570,1.000000,3.0,2.000000,1.000000,0.928570,0.928570,3.000000
3,0.000000,4.250000,95.000000,16.310000,1447.687500,0.937500,0.000000,294.154710,82.312500,92.687500,...,0.562500,1.000000,1.000000,1.000000,3.0,2.000000,1.000000,1.000000,1.000000,3.000000
4,0.000000,5.227270,95.000000,16.310000,1436.681820,0.909090,0.000000,304.983510,85.000000,96.000000,...,0.636360,1.000000,1.000000,1.000000,3.0,2.000000,1.000000,1.000000,0.954550,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,0.000000,4.907939,90.289337,14.149954,2336.116419,0.947393,0.000000,784.273429,77.781552,84.385586,...,0.000000,1.000000,1.000000,1.000000,7.0,0.000000,1.000000,1.000000,1.000000,3.000000
743,2.000000,4.000000,90.000000,16.160000,1540.000000,1.000000,0.000000,5732.461330,228.000000,233.000000,...,0.000000,1.000000,0.000000,1.000000,3.0,1.000000,1.000000,0.750000,1.000000,3.000000
744,0.000000,4.617650,96.000000,13.790000,1054.352940,0.676470,0.029410,920.665620,47.176470,48.088240,...,0.294120,0.970590,1.000000,1.000000,3.0,2.000000,0.000000,1.000000,0.941180,2.000000
745,0.403438,4.501075,93.000000,14.450000,1140.159818,0.900215,0.199570,2182.982620,628.000000,688.000000,...,0.000000,1.000000,1.000000,1.000000,3.0,2.000000,1.000000,1.000000,0.900215,3.000000


,Prevalencia
0,1.0
1,2.0
2,1.0
3,2.0
4,1.0
...,...
742,2.0
743,2.0
744,2.0
745,2.0


/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [30] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [30] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [30] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/

KeyboardInterrupt: 

In [ ]:
# Ordenar los resultados por accuracy de mayor a menor
results.sort(key=lambda x: x[2], reverse=True)

# Mostrar el mejor resultado
best_file,best_clasi, best_accuracy = results[0]
print(f"Mejor archivo: {best_file}, Mejor accuracy: {best_accuracy:.2f},Mejor Modelo: {best_clasi}")

In [ ]:
results=pd.DataFrame(results)
results.to_csv('/content/drive/MyDrive/Codigo Tesis Helicobacter Editado/Resultados/MLPClassifier.csv',index=False,sep=';',header=True )